# Rhythm tools

First, we'll create tools that actually work to do what we want here.

In [1]:
lyrics = {
    "title": "Fading Away",
    "intro": [
        "In the silence, I hear your voice",
        "A whispered reminder of our choice",
        "To let go, to move on, to lose",
        "The love we had, the memories we chose"
    ],
    "verse_one": [
        "We said forever, but forever's come and gone",
        "And now I'm left to face this empty dawn",
        "Your smile, a distant memory, a fleeting thought",
        "I'm searching for a way to heal my heart, to make it stop"
    ],
    "pre_chorus": [
        "But the pain remains, like an open sore",
        "A constant reminder of what we had before",
        "I'm trying to move on, but I'm stuck in this place",
        "Where love and loss collide, and my heart can't escape"
    ],
    "chorus": [
        "And I'm fading away, like a ghost in the night",
        "My heart is breaking, but it's not feeling right",
        "I'm trying to hold on, but you're slipping through my hands",
        "Leaving me with nothing but these tears and this pain"
    ],
    "verse_two": [
        "We said forever, but forever's just a lie",
        "And now I'm left to wonder why",
        "You walked away, without a fight",
        "Leaving me here, to face the dark of night"
    ],
    "outro": [
        "And I'll keep on fading, into the shadows of my mind",
        "Where love and loss entwine, and my heart will forever be left behind",
        "But maybe someday, I'll learn to let go",
        "And find my way back home, where love will set me free"
    ]
}

# Tool creation

In [2]:
import os
os.chdir('..')

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Lalasinha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Lalasinha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Divide into syllables tool

In [346]:
from typing import Dict
from smolagents import tool
from utils.syllable_split import get_syllable_dict
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag


def get_words(verse):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(verse)
    lower_words = [w.lower() for w in words]
    return lower_words


def get_keywords(verse):
    words = get_words(verse)
    tags = pos_tag(words)
    keyword_tags = [
        'JJ', 'RB', 'VB', 'NN', 'UH'
    ]
    to_be = [
        'am', 'im', 'm', 'are', 're', 'is', 's',
        'be', 'was', 'were'
    ]
    
    keywords = []
    for t in tags:
        match_keyword = any([
            a in t[1] for a in keyword_tags
        ]) and not any([
            be == t[0] for be in to_be
        ])
        if match_keyword and len(t[0]) > 1:
            keywords.append(t[0])
    
    return keywords


def get_similar_cut(word, cut):
    sim_cuts = {
        'k': {
            'gh': 'g',
            'cl': 'c',
            'cq': 'c',
            'cr': 'c',
            'ch': 'c',
            'cc': 'c',
            'ca': 'c',
            'co': 'c',
            'cu': 'c',
            'ci': 'c',
            'cy': 'c',
            'mc': 'm',
            'q': 'q' 
        },
        'e': {
            'ar$': 'a',
            'er$': 'e',
            'or$': 'o',
            'on$': 'o',
            'ir$': 'i',
            'ard$': 'a',
            'ors$': 'o',
            'ary$': 'a',
            'ation$': 'a',
            'war': 'w',
            'our': 'o',
            'abl': 'a',
            'ban': 'b',
            'aco': 'a',
            'ach': 'a',
            'ack': 'a',
            'ua': 'u',
            'ra': 'r',
            'ia': 'i',
            'ya': 'y',
            'iu': 'i',
        },
        's': {
            'ti': 't',
            'tual': 't',
            'ci': 'c',
            'ch': 'c',
            'cc': 'c',
            'ce': 'c',
            'ct': 'c',
            'cy': 'c',
            'zz': 'z',
            'x': 'x',    
        },
        'j': {
            'gi': 'g',
            'dge': 'd',
            'ge': 'g',
            'dua': 'd',
            'dul': 'd',
            'adu': 'a',
            'edu': 'e',
            'ndu': 'n',
            'rdu': 'r',
            'gy': 'g',
        },
        'z': {
            's': 's',
            'kz': 'k',
            'x': 'x',
        },
        'd': {
            'th': 't'
        },
        'f': {
            'ph': 'p'
        }
    }
    candidates_dict = sim_cuts.get(cut, None)
    if candidates_dict is not None:
        for key in candidates_dict.keys():
            if '$' in key:
                if word.endswith(key[:-1]):
                    return candidates_dict[key]
            else:
                if key in word:
                    return candidates_dict[key]
    return cut


def remove_trailing(word):
    if word[0] == '-':
        return word[1:]
    if word[-4:] == '----':
        return word[:-4]
    elif word[-3:] == '---':
        return word[:-3]
    elif word[-2:] == '--':
        return word[:-2]
    elif word[-1:] == '-':
        return word[:-1]
    else:
        return word


def process_syllables(word, phonemes, is_keyword):
    preprocessed_phonemes = [
        re.sub('0|2', '', re.sub(' ', '', p)) if is_keyword
        else re.sub('\d', '', re.sub(' ', '', p)) for p in phonemes
    ]
    accents = [
        True if '1' in p else False for p in preprocessed_phonemes
    ]
    cut_points = [
        p[0] for p in preprocessed_phonemes
    ]

    final_word = ''
    cut_word = word

    for i in range(1, len(accents)):
        cut = cut_points[i]
        was_accent = accents[i-1]
        if cut not in cut_word:
            try:
                prev_cut = cut_points[i-1]
                isolate_syl = ''.join(re.split(prev_cut, cut_word)[1:])
                cut = get_similar_cut(isolate_syl, cut)
            except:
                cut = get_similar_cut(cut_word, cut)

        splitted = re.split(cut, cut_word)
        rest = [cut + s for s in splitted[1:]]
        cut_word = ''.join(rest)

        if was_accent:
            splitted[0] = splitted[0].upper()
        final_word += splitted[0] + '-'
    
    if len(accents) > 0:
        if accents[-1]:
            cut_word = cut_word.upper()
    
    final_word += cut_word

    return remove_trailing(final_word)


@tool
def divide_syllables(verse: str, syllable_dict: Dict[str, str]) -> Dict[str, int]:
    """
    Divides a verse into syllables.

    Args:
        verse: A verse of a song.
        syllable_dict: the dictionary that maps words to its syllables.
    
    Returns:
        A dictionary with the verse text divided into syllables and syllable count.
    """
    words = get_words(verse)
    keywords = get_keywords(verse)
    phonemes = [syllable_dict.get(word, '') for word in words]
    syllables = ''
    
    for i in range(len(words)):
        is_keyword = words[i] in keywords
        if len(phonemes[i]) == 1:
            if is_keyword:
                words[i] = words[i].upper()
            syllables += words[i] + ' '
        else:
            syllables += process_syllables(
                words[i], phonemes[i], is_keyword
            ) + ' '
    syllables = syllables.strip()
    syllables_count = syllables.count(' ') + syllables.count('-') + 1
    return {
        'text': syllables,
        'syllables': syllables_count
    }

<>:146: SyntaxWarning: invalid escape sequence '\d'
<>:146: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Lalasinha\AppData\Local\Temp\ipykernel_22956\4269136863.py:146: SyntaxWarning: invalid escape sequence '\d'
  else re.sub('\d', '', re.sub(' ', '', p)) for p in phonemes


In [348]:
syllable_dict = get_syllable_dict()
verse_syllables = []

for section in list(lyrics.keys())[1:]:
    for verse in lyrics[section]:
        verse_syllables.append(divide_syllables(verse, syllable_dict))

verse_syllables

[{'text': 'in the SI-lence i HEAR your VOICE', 'syllables': 8},
 {'text': 'a WHI-spered re-MIN-der of our CHOICE', 'syllables': 9},
 {'text': 'to LET GO to MOVE on to LOSE', 'syllables': 8},
 {'text': 'the LOVE we HAD the memor-ies we CHOSE', 'syllables': 9},
 {'text': 'we SAID for-E-ver but for-E-ver s COME and GONE', 'syllables': 13},
 {'text': 'and NOW i m LEFT to FACE this EMP-ty DAWN', 'syllables': 11},
 {'text': 'your SMILE a DI-stant memory- a FLEE-ting THOUGHT',
  'syllables': 11},
 {'text': 'i m SEAR-ching for a WAY to HEAL my HEART to MAKE it STOP',
  'syllables': 15},
 {'text': 'but the PAIN re-MAINS like an O-pen SORE', 'syllables': 10},
 {'text': 'a CON-stant re-MIN-der of what we HAD be-fore', 'syllables': 12},
 {'text': 'i m TRY-ing to MOVE on but i m STUCK in this PLACE',
  'syllables': 14},
 {'text': 'WHERE LOVE and LOSS co-LLIDE and my HEART can t e-SCAPE',
  'syllables': 13},
 {'text': 'and i m FA-ding a-WAY like a GHOST in the NIGHT', 'syllables': 13},
 {'text': 'my

## Choose rythm tool

In [349]:
import numpy as np

def get_mean_syllables_per_stanza(stanza):
    syllable_dict = get_syllable_dict()

    stanza_syllables = []
    for verse in stanza:
        stanza_syllables.append(
            divide_syllables(verse, syllable_dict)
        )
    
    syllables_in_stanza = [
        verse['syllables'] for verse in stanza_syllables
    ]

    mean_syllables_per_stanza = np.round(
        np.mean(syllables_in_stanza)
    )

    return mean_syllables_per_stanza


def get_possible_time_signatures(stanza):
    mean_syllables = get_mean_syllables_per_stanza(stanza)
    signatures = {
        2: "2/4",
        3: "3/4",
        4: "4/4",
        6: "6/8",
        9: "9/8",
        12: "12/8"
    }

    answer = []
    for k in signatures.keys():
        if mean_syllables%k == 0:
            answer.append(signatures[k])
    
    return answer


def get_number_of_compasses(stanza, time_signature):
    mean_syllables = get_mean_syllables_per_stanza(stanza)
    verses = len(stanza)
    number_of_notes, type_of_notes = time_signature.split('/')
    note_type = int(int(type_of_notes)/4)
    compasses = round(mean_syllables/float(number_of_notes))
    
    print(compasses, 'compasses per verse')
    print(number_of_notes, '1/'+str(note_type), 'notes per compass')
    print(verses*compasses, 'compasses in total')

# Simple rythm agent

Now we have all the tools we need, we can create a simple rythm agent to test them.

In [4]:
import os
# os.chdir('..')
os.listdir()

['.git',
 '.gitignore',
 'env',
 'notebooks',
 'README.md',
 'requirements.txt',
 'src']

In [5]:
from smolagents import LiteLLMModel

smolagents_model = LiteLLMModel(
    model_id="ollama_chat/qwen2:7b",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)

c:\Users\Lalasinha\Documents\Python\llm-agents\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from src.modules import rhythm_tools
from smolagents import CodeAgent

agent = CodeAgent(
    tools=[
        rhythm_tools.get_syllable_dict_tool,
        rhythm_tools.divide_syllables,
        rhythm_tools.get_possible_time_signatures,
        rhythm_tools.get_number_of_compasses
    ],
    model=smolagents_model
)

answer = agent.run(
    """
    You are an experient songwriter. You will recieve the chorus of a song.
    Divide each verse in syllables.
    Chorus: {chorus}
    """.format(chorus=str(lyrics['chorus']))
)

# Then, choose one time signature from the possible time signatures.
#     Finally, answer the number of compasses that need to be created.

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an experient songwriter. You will recieve the chorus of a song.                                         │
│     Divide each verse in syllables.                                                                             │
│     Chorus: ["And I'm fading away, like a ghost in the night", "My heart is breaking, but it's not feeling      │
│ right", "I'm trying to hold on, but you're slipping through my hands", 'Leaving me with nothing but these tears │
│ and this pain'\]                                                                                                │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2:7b ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.APIConnectionError: Ollama_chatException - litellm.Timeout: Connection timed out after 600.0 seconds.

[Step 1: Duration 600.04 seconds]

AgentGenerationError: Error in generating model output:
litellm.APIConnectionError: Ollama_chatException - litellm.Timeout: Connection timed out after 600.0 seconds.